In [ ]:
suppressPackageStartupMessages({
    source(".Rprofile")
    library(hdf5r)
    library(rliger)
})

In [ ]:
PATH <- "e02_inmf"
if (!dir.exists(PATH)) {
    dir.create(PATH)
}

# Read data

In [ ]:
int.liger <- createLiger(list(
    atac = "e01_export_old_h5ad/atac2rna.h5ad",
    rna = "e01_export_old_h5ad/rna.h5ad"
), format.type = "AnnData")

In [ ]:
rna.var <- int.liger@raw.data$rna[["var"]][]
atac.var <- int.liger@raw.data$atac[["var"]][]
hvg <- rna.var$index[rna.var$highly_variable]
stopifnot(all(hvg %in% rna.var$index))
stopifnot(all(hvg %in% atac.var$index))

In [ ]:
hvg <- intersect(hvg, rna.var$index)
hvg <- intersect(hvg, atac.var$index)

In [ ]:
rna_cells <- int.liger@raw.data$rna[["obs"]][]$cell
atac_cells <- int.liger@raw.data$atac[["obs"]][]$cell

# Preprocessing

In [ ]:
int.liger <- normalize(int.liger)

In [ ]:
int.liger@var.genes <- hvg
int.liger <- scaleNotCenter(int.liger)

# Integration

In [ ]:
int.liger <- online_iNMF(int.liger, k = 20, miniBatch_size = 5000, seed = 0)
int.liger <- quantile_norm(int.liger, rand.seed = 0)

# Save results

In [ ]:
combined_latent <- int.liger@H.norm

In [ ]:
missing_cells <- setdiff(
    union(rna_cells, atac_cells),
    rownames(combined_latent)
)  # Because of cell filtering in scaleNotCenter
length(missing_cells)

In [ ]:
rna_latent <- combined_latent[rna_cells, ]
atac_latent <- combined_latent[atac_cells, ]
write.table(
    rna_latent, file.path(PATH, "rna_latent.csv"),
    sep = ",", row.names = TRUE, col.names = FALSE, quote = FALSE, na = ""
)
write.table(
    atac_latent, file.path(PATH, "atac_latent.csv"),
    sep = ",", row.names = TRUE, col.names = FALSE, quote = FALSE, na = ""
)